<h1>Análise Patrimonial das eleições </h1>



<h2>Proposta: Analisar a evolução patrimonial dos candidatos recorrentes na eleição de 2022 que se candidataram para as eleições de 2018 ou 2020</h2>


<h3>Pergunta Principal: Qual a evolução patrimonial dos candidatos recorrentes da eleição de 2022?</h3>

Aluno: **Juan Lima** [Linkedin](https://www.linkedin.com/in/juanrdlima/) [Email](mailto:juan.rdlima@gmail.com)

Esse projeto usará bases disponibilizadas pelo TSE (Tribunal Superior Eleitoral) sob licença de uso livre, disponibilizadas no site:<br>
[Portal de Dados Abertos](https://dadosabertos.tse.jus.br/)

Serão usados os arquivos relativos aos candidatos e aos respectivos bens declarados em cada eleição<br>

Arquivos Baixados no dia: 22/03/2025


**Link Individual de cada Arquivo utilizado:**
1. 2022 - Presidenciais [Candidatos](https://dadosabertos.tse.jus.br/dataset/candidatos-2022/resource/435145fd-bc9d-446a-ac9d-273f585a0bb9) [Bens](https://dadosabertos.tse.jus.br/dataset/candidatos-2022/resource/fac824ef-8519-4c75-b634-378e6fcc717f)

2. 2020 - Municipais [Candidatos](https://dadosabertos.tse.jus.br/dataset/candidatos-2020-subtemas/resource/8187b1aa-5026-4908-a15a-0bf777ee6701) [Bens](https://dadosabertos.tse.jus.br/dataset/candidatos-2020-subtemas/resource/4b5e016e-feed-4ff6-bf86-78217927709a)

3. 2018 - Presidenciais [Candidatos](https://dadosabertos.tse.jus.br/dataset/candidatos-2018/resource/d9cb832e-fa52-4b62-8ee3-8d68d5620116) [Bens](https://dadosabertos.tse.jus.br/dataset/candidatos-2018/resource/84475557-764f-4457-9277-92b58fbb5f80)



Para armazenamento e processamento foi utilizado a ferramenta DataBricks Community.

In [0]:
#Importação das Bibliotecas necessárias

#Muitas Bibliotecas já vem incorporadas no Databricks, vou listar as utilizadas que não precisaram ser importadas como comentário

#import dbutils

<h2>Importação dos Arquivos</h2>

Os arquivos disponibilizados nos links, estão no formato zip, para reduzir tempo de processamento realizei o download em minha máquina fiz o unzip e subi via upload no databricks, na pasta '/FileStore/tables/eleicoes', os zips contém 27 arquivos .csv separados em estados brasileiros e um com o compilado total, para essa análise usei os arquivos compilados.<br>

Nos zips também há um arquivo pdf com uma descrição resumo de cada coluna presente nas bases, disponível no repositório GIT



In [0]:
#Validando arquivos nas pastas
display(dbutils.fs.ls('/FileStore/tables/eleicoes'))

path,name,size,modificationTime
dbfs:/FileStore/tables/eleicoes/bem_candidato_2018_BRASIL.csv,bem_candidato_2018_BRASIL.csv,23257575,1743892826000
dbfs:/FileStore/tables/eleicoes/bem_candidato_2020_BRASIL.csv,bem_candidato_2020_BRASIL.csv,266192439,1743892854000
dbfs:/FileStore/tables/eleicoes/bem_candidato_2022_BRASIL.csv,bem_candidato_2022_BRASIL.csv,20810528,1743892863000
dbfs:/FileStore/tables/eleicoes/consulta_cand_2018_BRASIL.csv,consulta_cand_2018_BRASIL.csv,15929977,1743892867000
dbfs:/FileStore/tables/eleicoes/consulta_cand_2020_BRASIL.csv,consulta_cand_2020_BRASIL.csv,294252540,1743892889000
dbfs:/FileStore/tables/eleicoes/consulta_cand_2022_BRASIL.csv,consulta_cand_2022_BRASIL.csv,15746625,1743892895000


# Data Engineering

Irei separar o processo em três layers, cada um com seu respectivo schema.<br>
Irei mante-los no catálogo "padrão", pois o community não permite a criação de catalogos.

<h3>Bronze Layer:</h3><br> Dados Brutos convertidos em tabelas

In [0]:
%sql
--Criação Catalog
CREATE CATALOG IF NOT EXISTS  mvp1
    COMMENT 'Catalog for the mvp 1 (analise patrimonia de candidatos)' 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3002522921466276>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-3002522921466276>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("LS1Dcmlhw6fDo28gQ2F0YWxvZwpDUkVBVEUgQ0FUQUxPRyBJRiBOT1QgRVhJU1RTICBtdnAxCiAgICBDT01NRU5UICdDYXRhbG9nIGZvciB0aGUgbXZwIDEgKGFuYWxpc2UgcGF0cmltb25pYSBkZSBjYW5kaWRhdG9zKSc=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50     

In [0]:
%sql
--Criação Bronze Layer
CREATE SCHEMA IF NOT EXISTS  eleicao_bronze
    COMMENT 'Bronze Schema with raw tables' 

In [0]:
#Primeiro df
file_location = 'dbfs:/FileStore/tables/eleicoes/consulta_cand_2022_BRASIL.csv'
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"
encoding = "ISO-8859-1" #Encoding que funcionou melhor

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding",encoding) \
  .load(file_location)

df.limit(10).display()

DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,DS_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,SG_UF_NASCIMENTO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,544,Eleição Geral Federal 2022,02/10/2022,FEDERAL,BR,BR,BRASIL,1,PRESIDENTE,280001618036,22,JAIR MESSIAS BOLSONARO,JAIR BOLSONARO,#NULO,45317828791,NÃO DIVULGÁVEL,12,APTO,COLIGAÇÃO,22,PL,PARTIDO LIBERAL,-1,#NULO,#NULO,#NULO,280001682069,Pelo bem do Brasil,PP / REPUBLICANOS / PL,SP,21/03/1955,015564190337,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),01,BRANCA,272,PRESIDENTE DA REPÚBLICA,6,2º TURNO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,544,Eleição Geral Federal 2022,02/10/2022,FEDERAL,BR,BR,BRASIL,1,PRESIDENTE,280001644128,44,SORAYA VIEIRA THRONICKE,SORAYA THRONICKE,#NULO,60838965172,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,44,UNIÃO,UNIÃO BRASIL,-1,#NULO,#NULO,#NULO,280001683105,PARTIDO ISOLADO,UNIÃO,MS,01/06/1973,013250921929,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),01,BRANCA,276,SENADOR,4,NÃO ELEITO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,544,Eleição Geral Federal 2022,02/10/2022,FEDERAL,BR,BR,BRASIL,2,VICE-PRESIDENTE,280001677436,27,JOAO BARBOSA BRAVO,PROFESSOR BRAVO,#NULO,03532046768,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,27,DC,DEMOCRACIA CRISTÃ,-1,#NULO,#NULO,#NULO,280001684145,PARTIDO ISOLADO,DC,RJ,27/06/1947,045076570302,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),01,BRANCA,121,ECONOMISTA,4,NÃO ELEITO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,544,Eleição Geral Federal 2022,02/10/2022,FEDERAL,BR,BR,BRASIL,2,VICE-PRESIDENTE,280001600166,21,ANTONIO ALVES DA SILVA JUNIOR,ANTONIO ALVES,#NULO,04551706493,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,21,PCB,PARTIDO COMUNISTA BRASILEIRO,-1,#NULO,#NULO,#NULO,280001681036,PARTIDO ISOLADO,PCB,PE,07/10/1978,056283400876,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),02,PRETA,171,JORNALISTA E REDATOR,4,NÃO ELEITO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,AP,AP,AMAPÁ,3,GOVERNADOR,30001639521,28,GESIEL DE SOUZA OLIVEIRA,GESIEL DE OLIVEIRA,#NULO,61341509249,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,28,PRTB,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,-1,#NULO,#NULO,#NULO,30001682792,PARTIDO ISOLADO,PRTB,AP,03/02/1978,002462812585,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),01,BRANCA,999,OUTROS,4,NÃO ELEITO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,AP,AP,AMAPÁ,3,GOVERNADOR,30001620571,15,GILVAM PINHEIRO BORGES,GILVAM BORGES,#NULO,11907894268,NÃO DIVULGÁVEL,12,APTO,COLIGAÇÃO,15,MDB,MOVIMENTO DEMOCRÁTICO BRASILEIRO,-1,#NULO,#NULO,#NULO,30001682227,EFICIÊNCIA E TRABALHO,PROS / PODE / MDB,DF,01/08/1958,000346522577,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),03,PARDA,133,SOCIÓLOGO,4,NÃO ELEITO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,AP,AP,AMAPÁ,3,GOVERNADOR,30001643108,55,JAIME DOMINGUES NUNES,JAIME NUNES,#NULO,14673835204,NÃO DIVULGÁVEL,12,APTO,COLIGAÇÃO,55,PSD,PARTIDO SOCIAL DEMOCRÁTICO,-1,#NULO,#NULO,#NULO,30001683045,PRA MUDAR DE VERDADE,PTB / PSC / PROS / PSD / AGIR,AP,29/07/1959,000285072526,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),01,BRANCA,257,EMPRESÁRIO,4,NÃO ELEITO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,BA,BA,BAHIA,3,GOVERNADOR,50001604884,22,JOÃO INÁCIO RIBEIRO ROMA NETO,JOÃO ROMA,#NULO,81968442472

Código acima, usado para validar o encoding que conseguiu capturar melhor os dados, segundo algumas pesquisas e testes o "ISO-8859-1" ficou sendo a melhor<br> opção mesmo o arquivo de referência sugerir que o encoding do arquivo é "Latin1", como reparado infelizmente por causa do encoding todas as colunas vieram em formato de texto, que será tratado na Silver Layer a seguir.

In [0]:
#Criação da tabela permanente bronze de candidatos para teste

df.createOrReplaceTempView('candidatos')

permanent_table_name = "eleicao_bronze.candidatos"

schema = 'eleicao_bronze'

df.write.format("parquet").saveAsTable(permanent_table_name,mode='append')


In [0]:
%sql
SELECT 
  *
FROM 
  eleicao_bronze.candidatos
LIMIT 10

DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,DS_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,SG_UF_NASCIMENTO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,SP,SP,SÃO PAULO,7,DEPUTADO ESTADUAL,250001597754,30100,EMILIO CURY JUNIOR,EMILIO CURY,#NULO,18116335871,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,30,NOVO,PARTIDO NOVO,-1,#NULO,#NULO,#NULO,250001680946,PARTIDO ISOLADO,NOVO,SP,20/02/1973,180312170116,2,MASCULINO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),01,BRANCA,125,ADMINISTRADOR,5,SUPLENTE
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,SP,SP,SÃO PAULO,7,DEPUTADO ESTADUAL,250001597753,30333,EDSON TEIXEIRA,EDSON JAPÃO,#NULO,28750611860,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,30,NOVO,PARTIDO NOVO,-1,#NULO,#NULO,#NULO,250001680946,PARTIDO ISOLADO,NOVO,PR,01/07/1980,226994850167,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),01,BRANCA,125,ADMINISTRADOR,5,SUPLENTE
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,RS,RS,RIO GRANDE DO SUL,7,DEPUTADO ESTADUAL,210001620190,45610,NAASOM LUCIANO DA ROCHA,NAASOM LUCIANO,#NULO,00434866067,NÃO DIVULGÁVEL,12,APTO,FEDERAÇÃO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,1,Federação PSDB Cidadania,PSDB/CIDADANIA,PSDB/CIDADANIA,210001682211,FEDERAÇÃO,Federação PSDB Cidadania(PSDB/CIDADANIA),RS,24/12/1985,087268210434,2,MASCULINO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),03,PARDA,131,ADVOGADO,5,SUPLENTE
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,RS,RS,RIO GRANDE DO SUL,7,DEPUTADO ESTADUAL,210001649812,14678,MOACIR DA ROSA ALVES,DR MOACIR,#NULO,28035720015,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,14,PTB,PARTIDO TRABALHISTA BRASILEIRO,-1,#NULO,#NULO,#NULO,210001683289,PARTIDO ISOLADO,PTB,RS,27/07/1954,030247250906,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),01,BRANCA,115,ODONTÓLOGO,5,SUPLENTE
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,RS,RS,RIO GRANDE DO SUL,7,DEPUTADO ESTADUAL,210001596156,19020,ITACIR PEGORARO,PEGORARO,#NULO,43795005000,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,19,PODE,PODEMOS,-1,#NULO,#NULO,#NULO,210001680883,PARTIDO ISOLADO,PODE,RS,22/02/1966,065234100434,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),01,BRANCA,125,ADMINISTRADOR,5,SUPLENTE
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,BA,BA,BAHIA,7,DEPUTADO ESTADUAL,50001614687,15000,MARIA DE LOURDES TAVARES,LURDINHA,#NULO,66455731620,NÃO DIVULGÁVEL,3,INAPTO,PARTIDO ISOLADO,15,MDB,MOVIMENTO DEMOCRÁTICO BRASILEIRO,-1,#NULO,#NULO,#NULO,50001681743,PARTIDO ISOLADO,MDB,BA,21/05/1966,021022970566,4,FEMININO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),01,BRANCA,298,SERVIDOR PÚBLICO MUNICIPAL,-1,#NULO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,BA,BA,BAHIA,7,DEPUTADO ESTADUAL,50001605981,40004,ANTONIO JORGE DE BRITO,ANTONIO JORGE DE BRITO,#NULO,39869725520,NÃO DIVULGÁVEL,3,INAPTO,PARTIDO ISOLADO,40,PSB,PARTIDO SOCIALISTA BRASILEIRO,-1,#NULO,#NULO,#NULO,50001681308,PARTIDO ISOLADO,PSB,BA,07/08/1966,037599480582,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),02,PRETA,233,POLICIAL MILITAR,-1,#NULO
22/03/2025,03:36:11,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,

Próximos passo seria criar um loop para "appendar" os demais anos na mesma tabela, que irá facilitar o tratamento nas próximas camadas.<br>

Para facilitar esse processo criarei pastas para cada grupo de dados<br>

Para organização serão criadas duas tabelas dentro do Schema:
- Tabela de Candidatos: contendo as informações dos candidatos
- Tabela Bens: contendo os bens declarados pelos candidatos


In [0]:
#Criando pastas
nome_pastas = ['candidatos','bens']


for nomes in nome_pastas:
  dbutils.fs.mkdirs("dbfs:/FileStore/tables/eleicoes/"+nomes)


In [0]:
#Movendo arquivos para suas pastas
anos = ["2018","2020","2022"]

for ano in anos:
    dbutils.fs.mv(f"dbfs:/FileStore/tables/eleicoes/bem_candidato_{ano}_BRASIL.csv",f"dbfs:/FileStore/tables/eleicoes/bens/bem_candidato_{ano}_BRASIL.csv" )
    dbutils.fs.mv(f"dbfs:/FileStore/tables/eleicoes/consulta_cand_{ano}_BRASIL.csv",f"dbfs:/FileStore/tables/eleicoes/candidatos/consulta_cand_{ano}_BRASIL.csv" )


In [0]:
%sql
--Truncar tabela criada para executar looping
TRUNCATE TABLE  eleicao_bronze.candidatos

Tabela bronze Candidatos:

In [0]:
#Tabela bronze candidatos particionada por ano

folder = "dbfs:/FileStore/tables/eleicoes/candidatos"

df = (
    spark.read
    .option("recursiveFileLookup", "true")
    .format("binaryFile")
    .load(folder)
)

tabelas = []

tabelas.append(df.select('path').rdd.flatMap(lambda x: x).collect())

# display(tabelas)

for files in tabelas:
    
    file_type = "csv"

    # CSV options
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ";"
    encoding = "ISO-8859-1" #Encoding que funcionou

    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .option("encoding",encoding) \
      .load(files)

    df.createOrReplaceTempView('candidatos')

    permanent_table_name = "eleicao_bronze.candidatos"

    schema = 'eleicao_bronze'

    df.write.format("parquet").saveAsTable(permanent_table_name,mode='append')


In [0]:
%sql
--Verificar tabela criada
SELECT 
  *
FROM 
  eleicao_bronze.candidatos
LIMIT 10

DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,DS_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,SG_UF_NASCIMENTO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO
22/01/2025,15:04:00,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,71072,SÃO PAULO,13,VEREADOR,250001265966,27323,JOÃO BATISTA DOS SANTOS,JOÃO ZELADOR,#NULO#,13534893883,NÃO DIVULGÁVEL,3,INAPTO,PARTIDO ISOLADO,27,DC,DEMOCRACIA CRISTÃ,-1,#NULO#,#NULO#,#NULO#,250000136838,PARTIDO ISOLADO,DC,SP,05/03/1970,292412220132,2,MASCULINO,5,ENSINO MÉDIO INCOMPLETO,3,CASADO(A),03,PARDA,303,GERENTE,4,NÃO ELEITO
22/01/2025,15:04:00,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,71072,SÃO PAULO,13,VEREADOR,250001268518,28010,SILVIO ROBERTO VOCE,SILVIO VOCE,#NULO#,90421213868,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,28,PRTB,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,-1,#NULO#,#NULO#,#NULO#,250000125795,PARTIDO ISOLADO,PRTB,SP,15/04/1953,097094760159,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),01,BRANCA,182,COMISSÁRIO DE BORDO,4,NÃO ELEITO
22/01/2025,15:04:00,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,71072,SÃO PAULO,13,VEREADOR,250000925963,20901,ANDRESSA MACIEL DO NASCIMENTO,DEDE,#NULO#,37174723802,NÃO DIVULGÁVEL,3,INAPTO,PARTIDO ISOLADO,20,PSC,PARTIDO SOCIAL CRISTÃO,-1,#NULO#,#NULO#,#NULO#,250000106600,PARTIDO ISOLADO,PSC,SP,20/11/1988,344907730124,4,FEMININO,7,SUPERIOR INCOMPLETO,3,CASADO(A),03,PARDA,999,OUTROS,4,NÃO ELEITO
22/01/2025,15:04:00,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,PR,74888,TUNAS DO PARANÁ,13,VEREADOR,160001090215,28000,EDEMAR RIBEIRO BORGES,NICO DA MARACANÃ,#NULO#,62730614915,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,28,PRTB,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,-1,#NULO#,#NULO#,#NULO#,160000132022,PARTIDO ISOLADO,PRTB,PR,05/01/1965,004079340655,2,MASCULINO,3,ENSINO FUNDAMENTAL INCOMPLETO,1,SOLTEIRO(A),03,PARDA,541,MECÂNICO DE MANUTENÇÃO,4,NÃO ELEITO
22/01/2025,15:04:00,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,71072,SÃO PAULO,13,VEREADOR,250001100745,35020,ROSE LEILA RODASLI CHUERE,ROSE LEILA,#NULO#,53450779834,NÃO DIVULGÁVEL,3,INAPTO,PARTIDO ISOLADO,35,PMB,PARTIDO DA MULHER BRASILEIRA,-1,#NULO#,#NULO#,#NULO#,250000133375,PARTIDO ISOLADO,PMB,SP,11/01/1954,156241700167,4,FEMININO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),01,BRANCA,923,APOSENTADO (EXCETO SERVIDOR PÚBLICO),4,NÃO ELEITO
22/01/2025,15:04:00,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,70211,SANTA BRANCA,13,VEREADOR,250000984468,45000,JOSÉ AMILTON CORREA,BIKUIRA,#NULO#,26595781840,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,-1,#NULO#,#NULO#,#NULO#,250000115157,PARTIDO ISOLADO,PSDB,PR,17/08/1972,180429580124,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),01,BRANCA,707,TRABALHADOR METALÚRGICO E SIDERÚRGICO,5,SUPLENTE
22/01/2025,15:04:00,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,PR,75493,FAXINAL,13,VEREADOR,160000869858,17200,JOSÉ PINTO DE OLIVEIRA,ZÉ DE OLIVEIRA DO GÁS,#NULO#,10132848953,NÃO DIVULGÁVEL,12,APTO,PARTIDO ISOLADO,17,PSL,PARTIDO SOCIAL LIBERAL,-1,#NULO#,#NULO#,#NULO#,160000095987,PARTIDO ISOLADO,PSL,MG,25/06/1951,034885470612,2,MASCULINO,2,LÊ E ESCREVE,3,CASADO(A),01,BRANCA,169,COMERCIANTE,5,SUPLENTE
22/01/2025,15:04:00,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2

In [0]:
%sql
--Verificar se todos os arquivos foram inseridos
SELECT 
  DISTINCT ANO_ELEICAO
FROM 
  eleicao_bronze.candidatos

ANO_ELEICAO
2020
2018
2022


Tabela bronze Bens:

In [0]:
#Tabela dos Bens Candidatos

folder = "dbfs:/FileStore/tables/eleicoes/bens"

df = (
    spark.read
    .option("recursiveFileLookup", "true")
    .format("binaryFile")
    .load(folder)
)

tabelas = []

tabelas.append(df.select('path').rdd.flatMap(lambda x: x).collect())

# display(tabelas)

for files in tabelas:
    
    file_type = "csv"

    # CSV options
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ";"
    encoding = "ISO-8859-1" #Encoding que funcionou

    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .option("encoding",encoding) \
      .load(files)

    df.createOrReplaceTempView('bens_declarados')

    permanent_table_name = "eleicao_bronze.bens_declarados"

    schema = 'eleicao_bronze'

    df.write.format("parquet").saveAsTable(permanent_table_name,mode='append')

In [0]:
%sql
--Verificar tabela criada
SELECT 
  *
FROM 
  eleicao_bronze.bens_declarados
LIMIT 10

DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,NR_ORDEM_BEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULT_ATUAL_BEM_CANDIDATO,HH_ULT_ATUAL_BEM_CANDIDATO
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,22411,UMBUZEIRO,150001212623,1,13,Terreno,"TERRENO LOCALIZADO NO LOTEAMENTO DR. JOSÉ NIVALDO, SURUBIM-PE","76800,00",15/11/2020,22:05:27
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,20990,NATUBA,150001213783,1,13,Terreno,"PROPRIEDADE RURAL 60 HECTARE, NO SITIO COVÕES, NATUBA-PB","40000,00",15/11/2020,21:51:07
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,20990,NATUBA,150001213783,2,13,Terreno,"UM TERRENO E PRÉDIO NA AVENIDA DR FRANCISCO MONTENEGRO, 48, NO 1. PISO A MEDIDA 4M X 7M DE LARGURA, COM A CRIPTA PANTHEON DR. FRANCISCO MONTENEGRO E NO 2. PISO, MEDIDAS DE 7X7M","60000,00",15/11/2020,21:51:07
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,19658,CABEDELO,150001215952,2,19,Outros bens imóveis,"Apartamento em Construção de 60m² localizado na rua Bancário Antonio Rosa da Silva, S/N, Bancários - João Pessoa/PB","285000,00",15/11/2020,21:51:09
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,19658,CABEDELO,150001216673,1,13,Terreno,"1/3 do lote de terreno sob No. 07, quadra Y, Loteamnento Jardim Prolongado Jardim Atlantico - Cabedelo-PB","23333,33",15/11/2020,21:51:09
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,19658,CABEDELO,150001216692,1,19,Outros bens imóveis,"Sítio de aproximadamente 0,6 hectare, localizado no sitio Jardim, no município de Serraria, estado da Paraiba","40000,00",15/11/2020,21:51:09
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,19658,CABEDELO,150001216692,2,19,Outros bens imóveis,"Sítio de aproximadamente 0,6 hectare, localizado no sitio Jardim, no município de Serraria, estado da Paraiba","40000,00",15/11/2020,21:51:09
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,21512,QUEIMADAS,150001216696,4,13,Terreno,"Trinta lotes de terrenos, localizados no Loteamento Novo Horizonte, Queimadas/PB.","1350000,00",15/11/2020,21:47:46
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,21512,QUEIMADAS,150001216696,5,26,Linha telefônica,Telefonia fixa e móvel,"0,00",15/11/2020,21:47:46
22/01/2025,15:03:56,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,21512,QUEIMADAS,150001216730,1,19,Outros bens imóveis,"Propriedade rural com casa edificada, localizadas no Sítio Bodopitá, Queimadas/PB.","30000,00",15/11/2020,21:47:46


In [0]:
%sql
--Verificar se todos os arquivos foram inseridos
SELECT 
  DISTINCT ANO_ELEICAO
FROM 
  eleicao_bronze.bens_declarados
LIMIT 10

ANO_ELEICAO
09:38:20
2020
23:07:52
18/11/2020
22:14:31
22:52:49
17/11/2020
null
26/04/2024
14:11:23


Nesta parte acima pode-se notar que o arquivo de bens de 2022 possui linhas corrompidas, isso será ajustado na próxima layer:

------------------------------------------------------------------------------------------------------------